In [16]:
import csv
import glob
import shutil
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

# Create a folder with format (11-November 2018) in the directory 
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')
dir_name = 'S://TANF Reports//!2019 TANF Reports//'
os.chdir(dir_name)


folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Move all .csv files in Downloads folder to S:drive(destination) folder
path = 'C://Users//nnagaraj//Downloads'
os.chdir(path)
source = os.listdir(path)
destination = os.path.join('S://TANF Reports//!2019 TANF Reports', folder_name )

for files in source:  
    if files.endswith(".csv"):
        shutil.move(files, destination)  
        

# Convert all .csv files in S:drive to Excel        
os.chdir(destination)        
for filename in os.listdir(destination):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(destination, filename))
        #df['Job Ret Date'] = pd.to_datetime(df['Job Ret Date']) # for JRM Mailing & Universe
        #df['Begin Date'] = pd.to_datetime(df['Begin Date']) # for JRM Universe
        #df = df.sort_values(by = ['local office', 'Name'])
        #df = df.sort_values(by = ['local office', 'Last Name']) # for TEA Adults with Assigned Activity
        x = filename.replace("-", " ") + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title()
        writer = pd.ExcelWriter(str(x).title().replace(".Csv", "")+".xlsx")
        df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False, header = filename)
        writer.save()
        continue
    else:
        continue       

# Now that all csv have been converted to excel, delete all the csv files in the directory        
files = [file for file in glob.glob("*.csv") if not file.endswith("0.csv")]
for file in files:
    os.remove(file)

# Closure Due to Employment by County

### Merge Closure due to employment and Closure due to employment with address 

In [10]:
import csv
import pandas as pd
now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')

closure_emp = pd.read_excel('Closures Due To Employment - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')
closure_emp_add = pd.read_excel('Closures Due To Employment With Address - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')

df = pd.merge(closure_emp, closure_emp_add, how = 'inner', on = 'County')

df = df.drop(['SSN', 'Address 1', 'Address 2', 'Address 3', 'Zip'], axis = 1)


# At first use duplicated to check for duplicated values in your dataframe. Then select which columns you want to be changed
is_duplicated = df.apply(lambda x: x.duplicated())
df_map = df.where(~is_duplicated, " ")
df[['County', '# Closures Due to Employment', 'Total Closures', 'Percentage']] = df_map.drop(['Case No', 'Casehead Name'], axis = 1)
df.head(4)

# Write created df to excel
writer = pd.ExcelWriter('Closures Due to Employment by County' + ' - ' + prev.strftime('%B') + ' ' + now.strftime('%Y') + '.xlsx')
df.to_excel(writer, sheet_name = prev.strftime('%B'), index = False)#, header = filename 
writer.save()


# Delete file Closures Due To Employment - December 2018.xlsx
os.remove('Closures Due To Employment - ' + str_prev_month + ' ' + now.strftime('%Y') + '.xlsx')

In [25]:
"""
writer = pd.ExcelWriter('Closure due to Emp.xlsx')
df.to_excel(writer, 'Closure due to emp')
writer.save()
"""

In [ ]:
"""
is_duplicated = df.apply(lambda x : x.duplicated())
 df_map = df.where(~is_duplicated, '')
 df[['A','B']] = df_map.drop('C', axis=1)

##https://datascience.stackexchange.com/questions/36141/drop-duplicate-but-conserve-data-in-other-columns-with-pandas/37013#37013
>>> writer = pd.ExcelWriter('output.xlsx')
>>> df1.to_excel(writer,'Sheet1')
>>> df2.to_excel(writer,'Sheet2')
>>> writer.save()
"""

# Extracting Deferred, Exempt and Sanction from Disaggrigated file in S:drive

In [13]:
import os
import pandas as pd
import openpyxl
import datetime
from datetime import date, timedelta

now = datetime.datetime.now()
prev = date.today().replace(day=1) - timedelta(days=1)
int_prev_month = prev.month
str_prev_month = prev.strftime('%B')

folder_name = (str(int_prev_month) + ('-') + (str_prev_month) + ' ' + now.strftime('%Y')) #format ('11-November 2018')
destination = os.path.join('S://TANF Reports//!2019 TANF Reports', folder_name )


os.chdir(destination)
os.getcwd()


def riter(x):
    x.to_excel(writer, sheet_name = prev.strftime('%B'), index = False)
    writer.save()
    
df = pd.read_excel('Dissaggregated Def_Xmpt_Sanc By Local Office'+ ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')

    
sanction = df[df['Deferred / Exempt / Sanction'] == 'Sanction']
sanction = sanction.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Sanction by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')    
riter(sanction)

exempt = df[df['Deferred / Exempt / Sanction'] == 'Exempt']
exempt = exempt.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Exempt by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
riter(exempt)

deferred = df[df['Deferred / Exempt / Sanction'] == 'Deferred']
deferred = deferred.sort_values(by = ['Local Office'])
writer = pd.ExcelWriter('TEA Adults Deferred by Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
riter(deferred)

# Delete Disaggregated Def_Xmpt_Sanc By Local Office - June 2019.xlsx
if os.path.exists('Dissaggregated Def_Xmpt_Sanc By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx'):
    os.remove('Dissaggregated Def_Xmpt_Sanc By Local Office' + ' - ' + str_prev_month + ' ' + now.strftime('%Y').title() + '.xlsx')
else:
    print("The file does not exist")



# Section 3 (Aggregate)
### Delete 7 characters starting from T6 and delete any $ over 8 characters, then run the following code

In [11]:
from textwrap import wrap
t6_2018q4 = '000016120000155700001490000002800000025900000251000013320000129800001239$$$$$$$$000028750000282400002738000000640000006700000063000014810000146000001416000013300000129700001259000062960000619700006082000016150000160400001549000046810000459300004533000000010000000000000002000000010000000100000000000000010000000100000001000003650000032300000288TRAILER1'
#wrap(t6_2018q4,8)

In [1]:
import os
os.getcwd()

'C:\\Cobra\\ADWS\\py3env'

In [2]:
os.chdir('C:\\Cobra\\ADWS\\ARWINS')

In [3]:
os.getcwd()

'C:\\Cobra\\ADWS\\ARWINS'